# Import

In [1]:
import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd

# Repro & CUDA perf

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True   # speed
torch.backends.cudnn.deterministic = False  # ok avec benchmark

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)} | CUDA: {torch.version.cuda}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3060 Laptop GPU | CUDA: 12.1


# Transform

In [3]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.75, 1.0)),  # + robuste que simple Resize
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),  # éval stable
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# Dataset

In [4]:
# ====== Dataset + split (deux ImageFolder séparés) ======
root = "dataset/train"
full_for_split = datasets.ImageFolder(root)  # juste pour les indices
num_classes = len(full_for_split.classes)
print(f"Classes: {num_classes} -> {full_for_split.classes[:5]}{'...' if num_classes>5 else ''}")

train_size = int(0.9 * len(full_for_split))
val_size = len(full_for_split) - train_size
gen = torch.Generator().manual_seed(SEED)
train_subset, val_subset = random_split(full_for_split, [train_size, val_size], generator=gen)

# Crée DEUX datasets indépendants (même fichiers, transforms différents)
train_dataset = datasets.ImageFolder(root, transform=transform_train)
val_dataset   = datasets.ImageFolder(root, transform=transform_val)

# Applique indices du split
train_dataset = Subset(train_dataset, train_subset.indices)
val_dataset   = Subset(val_dataset,   val_subset.indices)

# ====== DataLoaders rapides ======
num_workers = min(8, os.cpu_count() or 2)
pin = device == "cuda"
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,
                          num_workers=num_workers, pin_memory=pin,
                          persistent_workers=(num_workers>0), prefetch_factor=4)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False,
                          num_workers=num_workers, pin_memory=pin,
                          persistent_workers=(num_workers>0), prefetch_factor=4)

Classes: 11 -> ['bread', 'dairy', 'dessert', 'egg', 'fried']...


# Entrainement + Modèle

In [ ]:
# ====== Modèle ======
weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
model = models.efficientnet_b0(weights=weights)
in_feats = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_feats, num_classes)

model.to(device)

# Mixed precision
scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))

# ====== Critère / Optim / Scheduler ======
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)  # AdamW + LR un peu plus haut
num_epochs = 20
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs)  # aligne T_max sur n_epochs

best_acc = 0.0
train_losses, val_losses, val_accuracies = [], [], []

# ====== Entraînement ======
for epoch in range(1, num_epochs+1):
    model.train()
    total_loss = 0.0
    for imgs, labels in train_loader:
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device=='cuda')):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    scheduler.step()
    avg_train_loss = total_loss / max(1, len(train_loader))

    # ====== Validation ======
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=='cuda')):
        for imgs, labels in val_loader:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = val_loss / max(1, len(val_loader))
    val_acc = correct / total

    # Logs
    print(f"Epoch {epoch:02d}/{num_epochs} | train_loss={avg_train_loss:.4f} "
          f"| val_loss={avg_val_loss:.4f} | val_acc={val_acc:.4f}")

    # Sauvegarde du meilleur
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            "model_state": model.state_dict(),
            "class_to_idx": full_for_split.class_to_idx,
            "epoch": epoch,
            "val_acc": best_acc
        }, "best_efficientnet_b0.pth")
        print(f"↑ Nouveau meilleur modèle (val_acc={best_acc:.4f})")

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

print(f"\nEntraînement terminé. Meilleure val_acc : {best_acc:.4f}")

# ====== Courbes ======
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Courbes de perte")
plt.xlabel("Époques"); plt.ylabel("Loss"); plt.legend()

plt.subplot(1,2,2)
plt.plot(val_accuracies, label="Val Acc")
plt.title("Courbe de précision")
plt.xlabel("Époques"); plt.ylabel("Accuracy"); plt.legend()
plt.tight_layout(); plt.show()

/tmp/ipykernel_519758/14675521.py:16: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=='cuda'))
/tmp/ipykernel_519758/14675521.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=='cuda')):
/tmp/ipykernel_519758/14675521.py:52: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=='cuda')):


Epoch 01/20 | train_loss=1.1044 | val_loss=0.8739 | val_acc=0.8564
↑ Nouveau meilleur modèle (val_acc=0.8564)
Epoch 02/20 | train_loss=0.8419 | val_loss=0.7440 | val_acc=0.9113
↑ Nouveau meilleur modèle (val_acc=0.9113)
Epoch 03/20 | train_loss=0.7699 | val_loss=0.7650 | val_acc=0.9015
Epoch 04/20 | train_loss=0.7263 | val_loss=0.7471 | val_acc=0.9053


KeyboardInterrupt: 

In [ ]:
# Collecte des vraies/predites sur le set de validation
model.eval()
all_preds, all_labels = [], []
with torch.no_grad(), torch.cuda.amp.autocast(enabled=(device=='cuda')):
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# Matrice de confusion
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(cm, display_labels=full_for_split.classes)
fig, ax = plt.subplots(figsize=(12, 10))
disp.plot(ax=ax, xticks_rotation=90, cmap="Blues", colorbar=False)
plt.title("Matrice de confusion - Validation")
plt.show()


In [ ]:
report = classification_report(all_labels, all_preds, target_names=full_for_split.classes, output_dict=True)
class_acc = {cls: report[cls]["precision"] for cls in report if cls in full_for_split.classes}

# Tri des classes selon performance
sorted_acc = sorted(class_acc.items(), key=lambda x: x[1])

# Graphique clair
plt.figure(figsize=(10, 6))
plt.barh([x[0] for x in sorted_acc], [x[1] for x in sorted_acc])
plt.xlabel("Précision par classe")
plt.ylabel("Classe")
plt.title("Performance du modèle par classe")
plt.tight_layout()
plt.show()


In [ ]:
# Récupérer les noms des classes depuis ImageFolder
class_names = full_for_split.classes

# Compter les indices d'appartenance par classe
train_counts = Counter([full_for_split.targets[i] for i in train_subset.indices])
val_counts   = Counter([full_for_split.targets[i] for i in val_subset.indices])

# Convertir en DataFrame pour lisibilité
df_counts = pd.DataFrame({
    "Classe": class_names,
    "Train": [train_counts[i] for i in range(len(class_names))],
    "Validation": [val_counts[i] for i in range(len(class_names))]
})

# Ajouter ratio train/val et total
df_counts["Total"] = df_counts["Train"] + df_counts["Validation"]
df_counts["% Train"] = (df_counts["Train"] / df_counts["Total"] * 100).round(1)
df_counts["% Val"] = (df_counts["Validation"] / df_counts["Total"] * 100).round(1)

# Afficher
print(df_counts.sort_values("Total", ascending=False).to_string(index=False))

# Visualisation rapide
plt.figure(figsize=(12, 6))
plt.bar(df_counts["Classe"], df_counts["Train"], label="Train", alpha=0.7)
plt.bar(df_counts["Classe"], df_counts["Validation"], label="Validation", alpha=0.7)
plt.title("Répartition des images par classe (Train vs Val)")
plt.xticks(rotation=45, ha="right")
plt.legend()
plt.tight_layout()
plt.show()
